In [ ]:
import numpy as np
import geopandas as gpd
from sklearn.cluster import KMeans

def select_seed_points(road_edges, n_clusters=4):
    # Try different quantiles to select low-accident roads
    for q in [0.3, 0.4, 0.5, 0.6, 0.7, 0.8]:  
        low_accident_roads = road_edges[road_edges['accident_count'] < road_edges['accident_count'].quantile(q)]
        if len(low_accident_roads) >= n_clusters:
            break

    # If still insufficient, select n_clusters roads with the fewest accidents
    if len(low_accident_roads) < n_clusters:
        low_accident_roads = road_edges.nsmallest(n_clusters, "accident_count")

    # Ensure selection is not empty
    if low_accident_roads.empty:
        raise ValueError("Unable to find low-accident roads. Please check the data or manually define seed points.")

    # Extract centroids of selected roads as candidate points
    low_accident_roads["centroid"] = low_accident_roads.geometry.centroid
    candidate_points = np.array([[p.x, p.y] for p in low_accident_roads["centroid"]])

    # Check if enough points exist for KMeans clustering
    if len(candidate_points) < n_clusters:
        raise ValueError(f"Not enough roads ({len(candidate_points)}) for KMeans clustering. Please adjust filter criteria.")

    # Run KMeans clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    kmeans.fit(candidate_points)
    seed_points = kmeans.cluster_centers_

    # Convert to GeoDataFrame
    seed_gdf = gpd.GeoDataFrame(geometry=gpd.points_from_xy(seed_points[:, 0], seed_points[:, 1]), crs=road_edges.crs)
    
    return seed_gdf

# Run seed point selection and plot
seed_gdf = select_seed_points(road_edges, n_clusters=4)
seed_gdf.plot(marker="*", color="red", markersize=100, alpha=0.7)
# Export seed points to Shapefile
seed_gdf.to_file("seed_points.shp", driver="ESRI Shapefile")


In [ ]:
from scipy.spatial import Voronoi
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, box

def generate_voronoi_with_boundary(seed_gdf, boundary):
    """
    Generate Voronoi diagram and clip to custom boundary
    
    Parameters:
    - seed_gdf: GeoDataFrame of selected seed points
    - boundary: user-defined bounding Polygon
    
    Returns:
    - GeoDataFrame of clipped Voronoi cells
    """
    points = np.array([[p.x, p.y] for p in seed_gdf.geometry])

    # Add extra points to avoid Voronoi calculation errors
    extra_points = np.array([
        [points[:, 0].min() - 1000, points[:, 1].min() - 1000],
        [points[:, 0].max() + 1000, points[:, 1].max() + 1000]
    ])
    all_points = np.vstack([points, extra_points])

    vor = Voronoi(all_points)

    voronoi_cells = []

    for region_idx in vor.regions:
        if not region_idx or -1 in region_idx:
            continue  # Skip invalid regions
        
        poly_points = [vor.vertices[i] for i in region_idx if i != -1]
        
        try:
            polygon = Polygon(poly_points)
            if polygon.is_valid:
                # Clip Voronoi cell to boundary
                clipped_polygon = polygon.intersection(boundary)
                voronoi_cells.append(clipped_polygon)
        except:
            continue  # Skip problematic geometries

    if len(voronoi_cells) == 0:
        raise ValueError("Voronoi generation failed. Please ensure seed points are well distributed.")

    voronoi_gdf = gpd.GeoDataFrame(geometry=voronoi_cells, crs=seed_gdf.crs)
    
    return voronoi_gdf

# Define boundary manually
boundary = box(420000, 430000, 450000, 470000)

# Run Voronoi computation and clipping
voronoi_gdf = generate_voronoi_with_boundary(seed_gdf, boundary)

# Plot Voronoi cells
fig, ax = plt.subplots(figsize=(8, 8))
voronoi_gdf.plot(ax=ax, alpha=0.3, edgecolor="blue", linewidth=1)
seed_gdf.plot(ax=ax, marker="*", color="red", markersize=100, alpha=0.7)
gpd.GeoSeries(boundary).plot(ax=ax, facecolor="none", edgecolor="black", linestyle="--")
plt.title("Voronoi Diagram (Clipped to Boundary)")
plt.show()


In [ ]:
import osmnx as ox
import networkx as nx
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import LineString

def find_marathon_route(seed_gdf, voronoi_gdf, target_distance=42000):
    """
    Generate marathon-length routes (~42km) starting from seed points within each Voronoi cell.
    """
    marathon_routes = []
    
    # Ensure seed points and Voronoi cells use WGS84 (EPSG:4326)
    if voronoi_gdf.crs is None or voronoi_gdf.crs.to_epsg() != 4326:
        voronoi_gdf = voronoi_gdf.to_crs(epsg=4326)
    if seed_gdf.crs is None or seed_gdf.crs.to_epsg() != 4326:
        seed_gdf = seed_gdf.to_crs(epsg=4326)

    for idx, seed in seed_gdf.iterrows():
        cell = voronoi_gdf.iloc[idx].geometry

        try:
            # Download OSM road network for the Voronoi cell
            G = ox.graph_from_polygon(cell, network_type="walk")
            if G is None:
                print(f"⚠️ No road network found in cell {idx}")
                continue
            
            # Project road network to UTM
            utm_crs = ox.projection.project_graph(G).graph["crs"]
            G = ox.project_graph(G, to_crs=utm_crs)

            # Project seed point to UTM
            seed_utm = gpd.GeoDataFrame(geometry=[seed.geometry], crs="EPSG:4326").to_crs(utm_crs)
            start_node = ox.distance.nearest_nodes(G, seed_utm.geometry.x.iloc[0], seed_utm.geometry.y.iloc[0])

        except Exception as e:
            print(f"⚠️ Failed to retrieve road network: {e}")
            continue

        # Search for closed loop routes near marathon distance
        for end_node in G.nodes:
            try:
                route = nx.shortest_path(G, start_node, end_node, weight="length")
                route_length = sum(ox.utils_graph.get_route_edge_attributes(G, route, "length"))

                if abs(route_length - target_distance) < 2000:  # Allow 2km margin
                    line_geom = LineString([(G.nodes[n]["x"], G.nodes[n]["y"]) for n in route])
                    marathon_routes.append(line_geom)
                    break
            except:
                continue  # Skip unreachable paths

    return gpd.GeoDataFrame(geometry=marathon_routes, crs="EPSG:4326")

# Run marathon route generation
marathon_routes_gdf = find_marathon_route(seed_gdf, voronoi_gdf)

# Plot marathon routes
fig, ax = plt.subplots(figsize=(10, 8))
voronoi_gdf.plot(ax=ax, alpha=0.3, edgecolor="blue", linewidth=1)
marathon_routes_gdf.plot(ax=ax, color="red", linewidth=2)
seed_gdf.plot(ax=ax, marker="*", color="black", markersize=100, alpha=0.7)
plt.title("Marathon Routes in Voronoi Cells")
plt.show()
